In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board
import matplotlib.pyplot as plt

import numpy as np

In [2]:
training_generator = DataLoader(64, "train")
val_generator = DataLoader(64, "val", shuffle=False)

In [3]:
lossHistory = []
valHistory = []
class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      pred = self.model.predict(val_generator)
      loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      loss = np.sum(loss) / len(loss)
      valHistory.append(loss)

      plt.plot(valHistory)
      plt.savefig("val.png", dpi=200)
      plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      print(" val_loss : {}".format(loss))

      self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [5]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [ ]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [ ]:
model = tf.keras.models.load_model("models/model-4-1089/")

In [7]:
#625/625 [==============================] - 76s 120ms/step - loss: 0.0710 - mae: 0.1957 - val_loss: 0.0967 - val_mae: 0.2318
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnueCallbacks()])

   998/109375 [..............................] - ETA: 12:03 - loss: 0.0923 - mae: 0.2423 val_loss : 0.09154156517510666


2022-03-30 19:16:17.920329: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-30 19:16:18.418018: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-03-30 19:16:18.431854: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-03-30 19:16:18.446492: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: models/model-1-915/assets
  1997/109375 [..............................] - ETA: 13:45 - loss: 0.0883 - mae: 0.2349 val_loss : 0.0889307796883222


2022-03-30 19:16:26.802229: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-03-30 19:16:26.816291: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: models/model-2-889/assets
  2995/109375 [..............................] - ETA: 14:08 - loss: 0.0862 - mae: 0.2308 val_loss : 0.08597961419265486
INFO:tensorflow:Assets written to: models/model-3-859/assets
  3993/109375 [>.............................] - ETA: 14:09 - loss: 0.0844 - mae: 0.2273 val_loss : 0.08696616370328521
INFO:tensorflow:Assets written to: models/model-4-869/assets
  4997/109375 [>.............................] - ETA: 14:04 - loss: 0.0828 - mae: 0.2243 val_loss : 0.08272026776836966
INFO:tensorflow:Assets written to: models/model-5-827/assets
  5994/109375 [>.............................] - ETA: 13:55 - loss: 0.0818 - mae: 0.2223 val_loss : 0.08281463973973228
INFO:tensorflow:Assets written to: models/model-6-828/assets
  7000/109375 [>.............................] - ETA: 13:51 - loss: 0.0809 - mae: 0.2205 val_loss : 0.08215452011691889
INFO:tensorflow:Assets written to: models/model-7-821/assets
  7993/109375 [=>.................

<Figure size 432x288 with 0 Axes>

In [18]:
indicies = get_halfkp_indeicies(Board("r3k1nr/pp2ppbp/n2p2p1/q1p5/3N1P2/1P2P3/PBPP3P/RN1K1B1R w kq - 1 9"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

[0.13856101]


In [8]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[-0.3675208] 1.0 r6r/pp2k1pp/2p1Bp2/2b1p3/4P3/8/PPPB1PPP/2KR3R b - - 0 14
[-0.24875295] -0.9999999999999999 r5k1/1b3ppp/p3p3/1pP2n2/8/5NP1/3R1PKP/5R2 b - - 1 24
[-0.28492737] 1.0 r2q1rk1/1b1pbppp/pp2p3/2p3n1/P1BnP1Q1/1P4P1/1BPP1P1P/R4RNK w - - 7 15
[-0.1943143] 1.0 8/p2Q2R1/7p/3pk2B/8/3P4/PPn2P1P/nN3K2 w - - 1 29
[-0.27632397] 1.0 rn1qk2r/pp2bppb/2p1p1np/3pP3/3P4/P2B1N1P/1PPB1PP1/RN1Q1RK1 b kq - 0 10
[-0.47265935] 1.0 rn1q1rk1/pp3b1p/2pb1p2/3p1Bp1/3Pp2P/2P1P3/PP1N1PP1/R1BQ1RK1 b - - 0 15
[-0.42469263] 1.0 2R5/5ppk/8/6p1/2p5/3p4/6rr/3R2K1 w - - 4 45
[-0.42500538] 1.0 1k3r2/p1p5/1p5P/8/6P1/5P2/8/4RRK1 b - - 0 35
[-0.26649833] 1.0 rn2r2k/pp1b2pp/3q1p2/2bPp3/2B4N/2P3B1/P4PPP/R2Q1R1K w - - 6 18
[-0.11016059] 1.0 4r1k1/p4p2/1p1p1bpp/2n5/8/2P2NPP/PP3PB1/R4K2 w - - 0 20
[-0.22793901] 1.0 rn1qkbnr/pp3ppp/2pp4/1B1Pp3/4P1b1/5N2/PPP2PPP/RNBQK2R w KQkq - 0 6
[0.39857173] 1.0 r3k1r1/2q1ppb1/2p5/p3p3/2P1P3/1PNQB1Pb/P4P1P/2KR2R1 b q - 1 22
[0.36378467] 1.0 4rrk1/p4p2/1pp3p1/4Pp1p/1P1P1R2/2q1BQ1P/P4R1K

IndexError: index 64 is out of bounds for axis 0 with size 64

In [9]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

INFO:tensorflow:Assets written to: production/assets
